# Il meccanismo di *dispatching*

Questa breve nota intende illustrare attraverso una serie di esempi il meccanismo di *dispatching* attraverso il quale viene dapprima selezionata la segnatura del metodo da invocare (durante la fase di compilazione) e quindi individuata l'implementazione da eseguire (durante la fase di esecuzione). 

Il caso più elementare è quello di una singola classe in cui ci sia un solo metodo con un certo nome (nell'esempio `f`).

In [135]:
public class Simple {
  public void f() {
    System.out.println("Simple::f");
  }
}

In queste circostanze, è evidente che il tipo *apparente* e *concerto* non possono che coincidere e che l'invocazoine di `f` su un oggetto di tipo `Simple` non può che produrre l'invocazione dell'unica implementazione esistente.

In [136]:
Simple s = new Simple();
s.f();

Simple::f


## Overloading

Se ci sono più metodi col medesimo nome, anche all'interno della stessa classe, ossia c'è un caso di *overloading*, la selezione della segnatura del metodo da invocare segue la logica del minor numero di conversioni (si cerca la segnatura detta *most specific*).

In [137]:
public class Overload {
  public void f(int x) {
    System.out.println("Overload::f(int)");
  }
  public void f(double x) {
    System.out.println("Overload::f(double)");
  }
}

In alcuni casi il numero di conversioni è 0 (come nelle prime due invocazioni seguenti), mentre in altri (la terza invocazione) è possibile effettuare 1 conversione (da `float` a `double`) e non c'è ambiguità.

In [138]:
Overload o = new Overload();
o.f(1);
o.f(1.0);  // 1.0 è un double, nessuna conversione
o.f(1.0f); // 1.0f è un float, una conversione

Overload::f(int)
Overload::f(double)
Overload::f(double)


Le cose si complicano se, date le segnature dei metodi definiti e l'invocazione da effettuare, esiste più di una segnatura che si adattarebbe alla chiamata a patto di fare uno stesso numero di conversioni.

In [139]:
public class Overload {
  public void f(int x, double y) {
    System.out.println("Overload::f(int, double)");
  }
  public void f(double x, int y) {
    System.out.println("Overload::f(double, int)");
  }
}

In [140]:
Overload o = new Overload();
o.f(1, 1);

CompilationException: 

In questo caso non è possibile individuare la segnatura *most specific* per cui il compilatore: data l'invocazione con due `int`, entrambe i metodi sono invocabili con 1 conversione, pertanto il compialtore non può scegliere quale segnatura selezione. Questo dipende dal tipo di invocazione, è evidente che le invocazioni che non causano conversioni, ad esempio, sono entrambe legittime:

In [141]:
o.f(1, 1.0);
o.f(1.0, 0);

Overload::f(int, double)
Overload::f(double, int)


## Ereditarietà

L'introduzione di un sottotipo apre la possibilità che su un oggetto di un sottotipo venga invocato un metodo definito per il supertipo.

In [142]:
public class Above {
  public void f() {
    System.out.println("Above::f");
  }
}

public class Below extends Above {
  public void g() {
    System.out.println("Below::f");
  }
}

Per prima cosa, osserviamo che in presenza di un sottotipo il tipo apparente e concreto non devono necessariamente coincidere, si aprono tre possibilità a seconda che il tipo apparente e concreto siano, rispettivamente:
* `Above` e `Above`,
* `Above` e `Below`,
* `Below` e `Below`

evidentemente il caso `Below` e `Above` non è possibile in quanto il secondo non è sottotipo del primo.

Nel primo caso è possibile solo l'invocazione di `f`

In [143]:
Above aa = new Above();
aa.f();

Above::f


ma non quella di `g`, dato non è definita per tale tipo

In [144]:
aa.g();

CompilationException: 

Nel secondo caso, è possibile invocare `f` perché è visibile (a partire dal tipo apparente) e la sua implementazione (nel tipo concreto) viene ereditata dal supertipo

In [145]:
Above ab = new Below();
ab.f();

Above::f


Però, sebbene il tipo concreto sia dotato di `g`, il compilatore deve scegliere il metodo sulla base del tipo apparente, quindi la chiamata di `g` resta impossibile.

In [146]:
ab.g();

CompilationException: 

Nel caso del sottotipo, invece, `g` diventa visibile (e `f` lo è perché ereditata), quindi sono possibili entrambe le invocazioni.

In [147]:
Below bb = new Below();
bb.f();
bb.g();

Above::f
Below::f


## Overriding

Il sottotipo può decidere di riscrivere l'implementazione di un metodo ereditato, ossia farne l'*overriding*

In [64]:
public class Above {
  public void f(double x) {
    System.out.println("Above::f(double)");
  }
}

public class Below extends Above {
  @Override
  public void f(double x) {
    System.out.println("Below::f(double)");
  }
}

Affinché ciò avvenga, è però necessario che il metodo riscritto abbia la medesima segnatura di quello nel supertipo. Se usiamo l'annotazione `@Override` il compilatore può aiutarci a scoprire che, poiché la segnatura è diversa, la nuova implementazione non è davvero un *override* (ma solo un *overload* come sarà chiarito nella prossima sezione)!

In [73]:
public class BelowErr extends Above {
  @Override
  public void f(int x) {
    System.out.println("BelowErr::f(int)");
  }
}

CompilationException: 

Tornando a considerare i tre possibili casi di combinazione tra tipo apparente e concreto

In [69]:
Above aa = new Above();
Above ab = new Below();
Below bb = new Below();

è ovvio che su `aa` e `bb` l'invocazione corrisponderà alle implementazioni definite nella classe del tipo (apparente e concreto):

In [71]:
aa.f(1.0);
bb.f(1.0);

Above::f(double)
Below::f(double)


Il caso interessante è quello in cui il tipo concreto sia il sottotipo; in tal caso (una volta che il compilatore ha determinato che la segnatura da chiamare è `f(double)`), l'inocazione riguarderà però il codice definito nel tipo apparente:

In [72]:
ab.f(1.0);

Below::f(double)


# Overloading ed ereditarietà

Il caso più complesso (e interessante) è quando si mescolano l'*overiding* e l'*overloading*, sopratutto quando quest'ultimo è determinato dall'ereditarietà (ossia quando un metodo del sottotipo effettua l'overloading di uno che è definito nel supertipo).

In [75]:
public class Above {
  public void f(double x) {
    System.out.println("Above::f(double)");
  }
}

In [81]:
public class Below extends Above {
  public void f(int x) {
    System.out.println("Below::f(int)");
  }
}

Come nel caso precedente, i casi in cui il tipo apparente coincide con quello concreto e non ci sono conversioni, sono banali:

In [88]:
Above aa = new Above();
Below bb = new Below();

aa.f(1.0);
bb.f(1);

Above::f(double)
Below::f(int)


Nel caso del supertipo, la chiamata con argomento `int` seleziona l'unico metodo (la cui segnatura è compatibile grazie ad una conversione)

In [92]:
aa.f(1);

Above::f(double)


In quello del sottotipo, la chiamata con argomento `double` seleziona il metodo ereditato (che non richiede conversioni)

In [98]:
bb.f(1.0);

Above::f(double)


La cosa si fa interessante se il tipo apparente non coincide con quello concreto. In tal caso, la chiamata con argomento `double` seleziona il metodo del tipo apparente con tale segnatura:

In [100]:
ab.f(1.0);

Above::f(double)


Cosa succede però con argomento `int`? Dal momento che il sottotipo ha un metodo che non ricihede conversioni, ci si potrebbe attendere che sia esso a venir chiamato.

In [103]:
ab.f(1);

Above::f(double)


Questo però non avviene perché il compilatore seleziona la segnatura sulla base del tipo apparente, per `Above` la segnatura selezionata è `f(double)` che è compatibile grazie ad una conversione.

Una volta selezionata la segnatura, l'invocazione selezionerà l'implementazione di un metodo di tale segnatura nel sottotipo; tale metodo non è definito nel sottotipo, ma è ereditato dal supertipo.

## Overriding e overloading (e ereditarità)

Avendo compreso i meccanismi che regolano i casi precedenti, non è difficile comprendere un caso in cui si presentino assieme tutte le possibilità.

In [107]:
public class Above {
  public void f(double x) {
    System.out.println("Above::f(double)");
  }
}

public class Below extends Above {
  public void f(int x) {
    System.out.println("Below::f(int)");
  }
  @Override
  public void f(double x) {
    System.out.println("Below::f(double)");
  }
}

Restano come sempre i casi banali (tipo apparente coincidente con il concreto, nessuna conversione):

In [118]:
Above aa = new Above();
Below bb = new Below();

aa.f(1.0);
bb.f(1);
bb.f(1.0);

Above::f(double)
Below::f(int)
Below::f(double)


Nel caso del supertipo, l'invocazione col tipo `int` sarà soddisfatta tramite una conversione

In [121]:
aa.f(1);

Above::f(double)


Nel caso in cui il tipo apparente è diverso dal concreto, quale che sia il tipo dell'argomento, verrà selezionata la segnatura `f(double)` che (in quest'ultimo caso grazie all'overriding) trova la sua implemetazione nel sottotipo:

In [123]:
Above ab = new Below();

ab.f(1);
ab.f(1.0);

Below::f(double)
Below::f(double)


Su `ab` non c'è verso di ottenere l'invocazione del metodo di `Below` con segnatura `f(int)`!